## Video Selection

Choose DAD videoclips based on Valence and Arousal scores

Created: Nov 18, 2025 <br>
Last modified: Nov 20, 2025

In [77]:
# Imports
import pandas as pd
import itertools

In [78]:
# Load data and round scores for sorting
df = pd.read_csv(r'C:\Users\aruizdargence\Downloads\LIRIS-ACCEDE-annotations\LIRIS-ACCEDE-annotations\annotations/ACCEDEranking.txt', sep='\t')

df["valenceValue_rounded"] = df["valenceValue"].round(2)
df["arousalValue_rounded"] = df["arousalValue"].round(2)

df

,id,name,valenceRank,arousalRank,valenceValue,arousalValue,valenceVariance,arousalVariance,valenceValue_rounded,arousalValue_rounded
0,0,ACCEDE00000.mp4,3755,787,2.764340,1.366938,0.118453,0.148906,2.76,1.37
1,1,ACCEDE00001.mp4,6711,6428,3.260093,2.792376,0.104211,0.154526,3.26,2.79
2,2,ACCEDE00002.mp4,5043,3591,3.047528,1.840160,0.109841,0.149310,3.05,1.84
3,3,ACCEDE00003.mp4,7584,5615,3.338461,2.470821,0.100953,0.164621,3.34,2.47
4,4,ACCEDE00004.mp4,5014,4463,3.042496,2.082151,0.110000,0.163321,3.04,2.08
...,...,...,...,...,...,...,...,...,...,...
9795,9795,ACCEDE09795.mp4,8315,3236,3.406446,1.754529,0.101927,0.141201,3.41,1.75
9796,9796,ACCEDE09796.mp4,9784,1775,3.591728,1.480602,0.163026,0.116369,3.59,1.48
9797,9797,ACCEDE09797.mp4,9729,1560,3.583121,1.450951,0.158730,0.118500,3.58,1.45
9798,9798,ACCEDE09798.mp4,9605,100,3.564251,1.321943,0.149619,0.203450,3.56,1.32


In [79]:
# create dictionary with 15 videos for each affect
affects = {
    "highV_highA" : [False, False],
    "highV_lowA" : [False, True],
    "lowV_highA" : [True, False],
    "lowV_lowA" : [True, True]
}

rows = []
minv = df.sort_values(by="valenceValue_rounded", ascending=True)[:500]
maxv = df.sort_values(by="valenceValue_rounded", ascending=False)[:500]

for label, (v_order, a_order) in affects.items():
    if v_order:
        subset = (minv.sort_values(by="arousalValue_rounded", ascending=a_order).head(100)[["name", "valenceValue_rounded", "arousalValue_rounded"]])

    else:
        subset = (maxv.sort_values(by="arousalValue_rounded", ascending=a_order).head(100)[["name", "valenceValue_rounded", "arousalValue_rounded"]])
    
    subset["path"] = "d:/optilab/synchrony/data/video"+ subset["name"]
    subset["name"] = subset["name"].str.replace(".mp4", "")
    subset ["affect"] = label

    rows.append(subset)

video_df = pd.concat(rows, ignore_index=True)
video_df

,name,valenceValue_rounded,arousalValue_rounded,path,affect
0,ACCEDE06618,3.59,4.49,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA
1,ACCEDE09013,3.52,4.40,d:/optilab/synchrony/data/videoACCEDE09013.mp4,highV_highA
2,ACCEDE07643,3.56,4.40,d:/optilab/synchrony/data/videoACCEDE07643.mp4,highV_highA
3,ACCEDE01212,3.59,4.36,d:/optilab/synchrony/data/videoACCEDE01212.mp4,highV_highA
4,ACCEDE06669,3.58,4.35,d:/optilab/synchrony/data/videoACCEDE06669.mp4,highV_highA
...,...,...,...,...,...
395,ACCEDE01926,1.55,2.15,d:/optilab/synchrony/data/videoACCEDE01926.mp4,lowV_lowA
396,ACCEDE02503,1.46,2.15,d:/optilab/synchrony/data/videoACCEDE02503.mp4,lowV_lowA
397,ACCEDE07983,1.54,2.18,d:/optilab/synchrony/data/videoACCEDE07983.mp4,lowV_lowA
398,ACCEDE06959,1.50,2.23,d:/optilab/synchrony/data/videoACCEDE06959.mp4,lowV_lowA


In [80]:
pairs = []

for v1, v2 in itertools.combinations(video_df.itertuples(index=False), 2):
    if v1.path == v2.path and v1.affect == v2.affect:
        condition = "same_name_same_affect"

    elif v1.path != v2.path and v1.affect == v2.affect:
        condition = "same_affect_diff_name"

    elif v1.path != v2.path and v1.affect != v2.affect:
        condition = "diff_name_diff_affect"
    
    else:
        continue

    pairs.append({
        "v1_name" : v1.path,
        "v1_affect" : v1.affect,
        "v2_name" : v2.path,
        "v2_affect" : v2.affect,
        "condition" : condition
    })

pairs_df = pd.DataFrame(pairs)
pairs_df

,v1_name,v1_affect,v2_name,v2_affect,condition
0,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE09013.mp4,highV_highA,same_affect_diff_name
1,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE07643.mp4,highV_highA,same_affect_diff_name
2,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE01212.mp4,highV_highA,same_affect_diff_name
3,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE06669.mp4,highV_highA,same_affect_diff_name
4,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE01207.mp4,highV_highA,same_affect_diff_name
...,...,...,...,...,...
79795,d:/optilab/synchrony/data/videoACCEDE02503.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE06959.mp4,lowV_lowA,same_affect_diff_name
79796,d:/optilab/synchrony/data/videoACCEDE02503.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE01231.mp4,lowV_lowA,same_affect_diff_name
79797,d:/optilab/synchrony/data/videoACCEDE07983.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE06959.mp4,lowV_lowA,same_affect_diff_name
79798,d:/optilab/synchrony/data/videoACCEDE07983.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE01231.mp4,lowV_lowA,same_affect_diff_name


In [81]:
video1 = video_df
while True:
    video2 = video1.sample(frac=1).reset_index(drop=True)
    
    # ensure no video is paired with itself
    if not (video1["name"] == video2["name"]).any():
        break


pairs = pd.DataFrame({
    "video1_name": video1["path"],
    "video1_affect": video1["affect"],
    "video2_name": video2["path"],
    "video2_affect": video2["affect"]
})

# Add condition column
def get_condition(row):
    if row.video1_name == row.video2_name and row.video1_affect == row.video2_affect:
        return "same_name_same_affect"
    elif row.video1_affect == row.video2_affect and row.video1_name != row.video2_name:
        return "same_affect_diff_name"
    elif row.video1_affect != row.video2_affect:
        return "diff_affect_diff_name"
    else:
        return "other"

pairs["condition"] = pairs.apply(get_condition, axis=1)

# save to csv
pairs.to_csv(r'd:\optilab\synchrony\data/paired_videos.csv')

pairs


,video1_name,video1_affect,video2_name,video2_affect,condition
0,d:/optilab/synchrony/data/videoACCEDE06618.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE01397.mp4,lowV_highA,diff_affect_diff_name
1,d:/optilab/synchrony/data/videoACCEDE09013.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE01228.mp4,lowV_highA,diff_affect_diff_name
2,d:/optilab/synchrony/data/videoACCEDE07643.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE02093.mp4,lowV_highA,diff_affect_diff_name
3,d:/optilab/synchrony/data/videoACCEDE01212.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE07140.mp4,highV_highA,same_affect_diff_name
4,d:/optilab/synchrony/data/videoACCEDE06669.mp4,highV_highA,d:/optilab/synchrony/data/videoACCEDE00794.mp4,lowV_highA,diff_affect_diff_name
...,...,...,...,...,...
395,d:/optilab/synchrony/data/videoACCEDE01926.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE00025.mp4,lowV_highA,diff_affect_diff_name
396,d:/optilab/synchrony/data/videoACCEDE02503.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE03431.mp4,lowV_highA,diff_affect_diff_name
397,d:/optilab/synchrony/data/videoACCEDE07983.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE06509.mp4,highV_highA,diff_affect_diff_name
398,d:/optilab/synchrony/data/videoACCEDE06959.mp4,lowV_lowA,d:/optilab/synchrony/data/videoACCEDE09031.mp4,highV_lowA,diff_affect_diff_name
